# Jupyter Notebook Exploring Heart Failure Clinical Data


## Summary Statistics

Before we do anything else, let's import the data into R. Here we'll just download the data directly from [kaggle](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data?resource=download)
.

In [ ]:
med_data <-read.csv('https://raw.githubusercontent.com/dcolinmorgan/r-jupyterlite-website/main/lab/workspaces/heart_failure_clinical_records_dataset.csv')

## dataframe

In [ ]:
head(med_data)

Now that the data is loaded into R, let's show some statistical properties of the data using `summary()`.

In [ ]:
summary(med_data)

In [ ]:
dim(med_data)

## Data Visualisation and Exploration

The table of results above gives us a good numerical summary of the data, but it looks a little dry. Let's try creating something a little more interesting visually.

### Pairs Plot Visual Summary

We'll create a visual summary using the `pairs()` plotting function. This will create a matrix of scatter plots showing some of the measurements in the dataset plotted against one another. We'll also set the colour of the points using the age group of each subject.

In [ ]:
med_data$sex[med_data$sex == 0] <- "female"
med_data$sex[med_data$sex == 1] <- "male"

In [ ]:
col <- c("red", "yellow")
names(col) <- c("male", "female")
pairs(med_data[, c(3,5,7,8,9)], main="medical dataset", pch=21, bg=col[med_data$sex])

In [ ]:
head(med_data)

subset columns

In [ ]:
med_data[0:10,c('platelets','sex')] ### rows then column name

math operations between numeric columns

In [ ]:
mean(med_data$platelets[med_data$sex == 'male'])

In [ ]:
mean(med_data$platelets[med_data$sex == 'female'])

In [ ]:
(t.test(platelets ~ sex, data = med_data, var.equal=TRUE))

### Subject ejection_fraction Histograms 
Let's create some plots of subject platelet in the form of a histogram. We'll use faceting to make several plots at once, so we can see the how the data looks spead over the different age groups of subject.

In [ ]:
hist(med_data$ejection_fraction[med_data$sex == 'male'],breaks=4)

In [ ]:
hist(med_data$ejection_fraction[med_data$sex == "female"],breaks=20)

In [ ]:
hist(x = med_data$ejection_fraction[med_data$sex == 'male'],
     main = "Two Histograms in one",
     xlab = "age",
     ylab = "Frequency",
     breaks = 30,
     xlim = c(10, 90),
     col = "red"
     )

hist(x = med_data$ejection_fraction[med_data$sex == 'female'],
     breaks = 30,
     xlim = c(10, 90),
     add = TRUE, # Add plot to previous one!
      col = "yellow"
     )

In this dataset it looks like old subjects might have lower overall ejection fraction on average than the other age groups. Let's see if we can formalise something along those lines with a statistical test.

We'll use a T-test to see if there is a difference between average ejection fraction between age groups of subject.

In [ ]:
(t.test(ejection_fraction ~ sex, data = med_data, var.equal=TRUE))

## Fitting a model using Logistic Regression

Let's pretend a subject super-fan has seen the plots we made in the previous section. After seeing the clustering in the first pairs plot they suspect that it may be possible to predict the livign status of a subject based on medical measurements. They then ask us if we would be able to build a predictive model to find "alive" subjects based on the measurements.

Since the outcome variable here is logical, we choose to perform a logistic regression. First, let's split the data into training and validation sets, and create a new indicatior column for subjects which have died subjects.

In [ ]:
head(med_data)

In [ ]:
set.seed(0)
N <- nrow(med_data)
idx <- sample(1:N, N*2/3)
train <- med_data[idx,]
validation <- med_data[-idx,]

Next, we'll fit a logistic regression (logit) model using R's `glm()` function,

In [ ]:
train$dead = train$DEATH_EVENT == "1"
model <- glm(dead ~ creatinine_phosphokinase + ejection_fraction + platelets + serum_creatinine + serum_sodium,
             data = train, family=binomial(link="logit"))
summary(model)

### Assessing Model Predictions

We now have a model that uses penguin measurements to predict if that subject is a alive. Let's try it out using our validation set,

In [ ]:
prediction = ifelse(predict(model, validation, type = "response") < 0.5, "dead", "not dead")
table_mat=table(prediction, validation$DEATH_EVENT)

## how to tell if this is good prediction?
every metric tells a different story and depends on which your study is interested in

accuracy = (TN + TP)/(TP + TN + FP + FN)

[adapted from here](https://www.guru99.com/r-generalized-linear-model.html#9)

In [ ]:
accuracy_Test <- sum(diag(table_mat)) / sum(table_mat)

## line by line operations


accuracy_Test

In [ ]:
precision <- function(matrix) {
	# True positive
    tp <- matrix[2, 2]
	# false positive
    fp <- matrix[1, 2]
    return (tp / (tp + fp))
}

recall <- function(matrix) {
# true positive
    tp <- matrix[2, 2]# false positive
    fn <- matrix[2, 1]
    return (tp / (tp + fn))
}

In [ ]:
prec <- precision(table_mat)
prec
rec <- recall(table_mat)
rec

In [ ]:
f1 <- 2 * ((prec * rec) / (prec + rec))
f1